<a href="https://colab.research.google.com/github/JeroAgullo/fundamental_valuation/blob/main/fundamental_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Fundamental Valuation using Python

## 1.1. Introduction

**Project Approach**  
This project provides a fundamental valuation of NVIDIA Corporation (NVDA) using Python tools and financial data sourced from the Yahoo Finance API via the **yfinance** library.

The notebook parses and standardizes key financial statement elements into a format that facilitates **ratio analysis and valuation modeling**.

**Fundamental Analysis**  
Fundamental analysis is a method used in accounting and finance to estimate the **intrinsic value of a security** in order to help with investment decisions.

Unlike technical analysis, which focuses on price trends, fundamental analysis seeks to determine the "true" value of a security to identify whether it is overvalued, undervalued, or fairly valued **compared with its current market price**.

## 1.2. First steps

We will start by importing the needed libraries. Then, we are creating an object for the chosen company using yfinance's function '.Ticker', allowing us to get all its public financial information.

We are using NVIDIA's ticker (the market "ID") for this project, but any other ticker can be applied. Here is a list of companies in the S&P500 Index, and their ticker: https://en.wikipedia.org/wiki/List_of_S%26P_500_companie

In [82]:
# Libraries import
import yfinance as yf
import pandas as pd

In [83]:
# NVIDIA object creation
ticker = yf.Ticker("NVDA")

In [84]:
# Connection check
headers = ticker.financials.index.tolist() # We will print the headers of the Income Statement report
for h in headers:
  print(h)

Tax Effect Of Unusual Items
Tax Rate For Calcs
Normalized EBITDA
Total Unusual Items
Total Unusual Items Excluding Goodwill
Net Income From Continuing Operation Net Minority Interest
Reconciled Depreciation
Reconciled Cost Of Revenue
EBITDA
EBIT
Net Interest Income
Interest Expense
Interest Income
Normalized Income
Net Income From Continuing And Discontinued Operation
Total Expenses
Total Operating Income As Reported
Diluted Average Shares
Basic Average Shares
Diluted EPS
Basic EPS
Diluted NI Availto Com Stockholders
Net Income Common Stockholders
Net Income
Net Income Including Noncontrolling Interests
Net Income Continuous Operations
Tax Provision
Pretax Income
Other Income Expense
Other Non Operating Income Expenses
Special Income Charges
Restructuring And Mergern Acquisition
Net Non Operating Interest Income Expense
Interest Expense Non Operating
Interest Income Non Operating
Operating Income
Operating Expense
Research And Development
Selling General And Administration
Gross Profit

## 1.3. Data extract

We are now getting all the necessary data to proceed with the fundamental analysis. Not all the accessible information is needed, so we are selecting specific one from each Financial State (Financials):

- Income Statement
- Balance Sheet
- Cashflow

The market information (such as price or number of shares) will be also consulted for the more advanced financial valuations.

In [85]:
# Auxiliar function to get only existing values and to limit error cases
def safe_extract(df, row_name, column_name):
    if row_name in df.index:
        return df.loc[row_name, column_name]
    else:
        return None

In [86]:
# Data extract - Income Statement
df = ticker.financials
column_name = df.columns[0] # First column, to get TTM (Trailing Twelve Months) report

total_revenue = safe_extract(df, "Total Revenue", column_name)
gross_profit = safe_extract(df, "Gross Profit", column_name)
ebitda = safe_extract(df, "EBITDA", column_name)
ebit = safe_extract(df, "EBIT", column_name)
net_income = safe_extract(df, "Net Income", column_name)

In [87]:
# Data extract - Balance Sheet
df= ticker.balance_sheet
column_name = df.columns[0] # First column, to get only the latest accounting period report

total_assets = safe_extract(df, "Total Assets", column_name)
current_assets = safe_extract(df, "Current Assets", column_name)
inventory = safe_extract(df, "Inventory", column_name)
total_liabilities = safe_extract(df, "Total Liabilities Net Minority Interest", column_name)
current_liabilities = safe_extract(df, "Current Liabilities", column_name)
total_equity = safe_extract(df, "Total Equity Gross Minority Interest", column_name)
cash = safe_extract(df, "Cash And Cash Equivalents", column_name)
ordinary_shares = safe_extract(df, "Ordinary Shares Number", column_name)

In [88]:
# Data extract - Cashflow
df = ticker.cashflow
column_name = df.columns[0] # First column, to get TTM (Trailing Twelve Months) report

free_cashflow = safe_extract(df, "Free Cash Flow", column_name)

In [103]:
# Data extract - Market information
df = ticker.info

current_price = df["currentPrice"]
current_shares = df["sharesOutstanding"]
current_market_cap = df["marketCap"]

In [105]:
# Results print
print("Income Statement results")
print(f"Total Revenue: {total_revenue:,.0f}$")
print(f"Total Profit: {gross_profit:,.0f}")
print(f"EBITDA: {ebitda:,.0f}$")
print(f"EBIT: {ebit:,.0f}$")
print(f"Net Income: {net_income:,.0f}$")

print("\nBalance Sheet results")
print(f"Total Assets: {total_assets:,.0f}$")
print(f"Total Liabilities: {total_liabilities:,.0f}$")
print(f"Total Equity: {total_equity:,.0f}$")
print(f"Cash: {cash:,.0f}$")

print("\nMarket information")
print(f"Current Price: {current_price:,.2f}$")
print(f"Current Shares: {current_shares:,.0f}")
print(f"Current Market Cap: {current_market_cap:,.0f}$")

Income Statement results
Total Revenue: 130,497,000,000$
Total Profit: 97,858,000,000
EBITDA: 86,137,000,000$
EBIT: 84,273,000,000$
Net Income: 72,880,000,000$

Balance Sheet results
Total Assets: 111,601,000,000$
Total Liabilities: 32,274,000,000$
Total Equity: 79,327,000,000$
Cash: 0$

Market information
Current Price: 173.72$
Current Shares: 24,387,600,384
Current Market Cap: 4,236,614,041,600$


# 2. Financial Ratios

## 2.1. Introduction

Financial ratios are quantitative metrics derived from Financial Statements' data, allowing us to simplify the interpretation of such raw information.

These ratios will gives us a clear data-driven picture of the studied security and its past and current health, being a key part of fundamental valuation. They can be structured as followed:

- Leverage and Liquidity Ratios
- Profitability Ratios
- Valuation Ratios (Multiples)

## 2.2. Leverage and Liquidity Ratios

Indicators that let us know the company's the strength of its capital structure and the capacity to attend to its long-term (leverage) and its short-term obligations (liquidity).

In [91]:
# Leverage Ratios
debt_equity = total_liabilities / total_equity
print(f"Debt to Equity (D/E) Ratio: {debt_equity:.2%}")

debt_asset = total_liabilities / total_assets
print(f"Debt to Asset (D/A) Ratio: {debt_asset:.2%}")

Debt to Equity (D/E) Ratio: 40.68%
Debt to Asset (D/A) Ratio: 28.92%


In [92]:
# Liquidity Ratios
current = current_assets / current_liabilities
print(f"Current Ratio: {current:.2%}")

quick = (current_assets - inventory) / current_liabilities
print(f"Quick (Acid) Ratio: {quick:.2%}")

cash = cash / current_liabilities
print(f"Cash Ratio: {cash:.2%}")

Current Ratio: 443.99%
Quick (Acid) Ratio: 388.13%
Cash Ratio: 47.59%


## 2.3. Profitability Ratios

Indicators that let us know, not only if the company can attend to its obligations, but also if it gives profit to its owners.

In [93]:
# Return Ratios
roi = net_income / total_revenue
print(f"Return on Investment (ROI): {roi:.2%}")

roe = net_income / total_equity
print(f"Return on Equity (ROE): {roe:.2%}")

roa = net_income / total_assets
print(f"Return on Assets (ROA): {roa:.2%}")

Return on Investment (ROI): 55.85%
Return on Equity (ROE): 91.87%
Return on Assets (ROA): 65.30%


In [94]:
# Efficency Ratios
gross_margin = gross_profit / total_revenue
print(f"Gross Margin: {gross_margin:.2%}")

ebitda_margin = ebitda / total_revenue
print(f"EBITDA Margin: {ebitda_margin:.2%}")

fcf_revenue = free_cashflow / total_revenue
print(f"Free Cash Flow Margin: {fcf_revenue:.2%}")

Gross Margin: 74.99%
EBITDA Margin: 66.01%
Free Cash Flow Margin: 46.63%


In [97]:
# Share Ratios
book_value = total_equity / ordinary_shares # We use the share number reported in the same period as the Equity (Accounting Period)
print(f"Book Value per Share (BV): {book_value:.2f}$")

eps = net_income / current_shares # We use the share number reported in the same period as the Net Income (TTM)
print(f"Earnings per Share (EPS): {eps:.2f}$")

Book Value per Share (BV): 3.24$
Earnings per Share (EPS): 2.99$


## 2.4. Valuation Ratios (Multiples)

Indicators most commonly used in financial investments. They allow us to compare companies from the same sector and competitors in the market.

In [99]:
# Price to Book Value
p_bv = current_price / book_value
print(f"Price to Book Value (P/BV): x{p_bv:.2f}")

Price to Book Value (P/BV): x53.60


In [100]:
# Price to Earnings Ratio (PER)
per = current_price / eps
print(f"Price to Earnings Ratio (PER or P/E): x{per:.2f}")

Price to Earnings Ratio (PER): x58.13


In [106]:
# Enterprise Value to EBITDA (EV/EBITDA)
enterprise_value = current_market_cap + total_liabilities - cash # Enterprise Value (EV) must be calculated first
ev_ebitda = enterprise_value / ebitda
print(f"Enterprise Value to EBITDA (EV/EBITDA): x{ev_ebitda:.2f}")

Enterprise Value to EBITDA (EV/EBITDA): x49.56
